In [1]:
import pandas as pd
import numpy as np

# Importing the dataset
salary_data = pd.read_csv('kaggle_survey_2020_responses.csv')

## Q11,12,14 and 15

#### New DataFrame containg just the columns corresponding to these questions

In [2]:
df_1 = salary_data.iloc[:,47:52]
df_2 = salary_data.iloc[:,53:66]

df_q11_12_14_15 = pd.concat([df_1, df_2], axis = 1)

#### Cleaning and encoding

### Q11

In [3]:
print(f"The column has {df_q11_12_14_15['q11'].isnull().sum()} null values")

The column has 3007 null values


When we observe the data and the response choices for this question, it is evident that null values (associated with no response - empty entry in the datasheet) imply that the person didn't specify any platform. Thus, we shall fill these null values with 'mode' for the column.

In [4]:
df_q11_12_14_15['q11'].fillna(df_q11_12_14_15['q11'].mode()[0], inplace = True)

Distribution of the column:

In [5]:
df_q11_12_14_15['q11'].value_counts()

A personal computer or laptop                                          16355
A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc)     2358
A deep learning workstation (NVIDIA GTX, LambdaLabs, etc)                834
None                                                                     292
Other                                                                    197
Name: q11, dtype: int64

#### Method for encoding

##### Motivation
Since this column has 5 categories we could one-hot encode this column but since we are dealing with multiple questions which have several unique categories, plus some of the observations in those columns have multiple categories as their value thus we would have to resort to one-hot encoding for such columns, thus, we believe that it would be reasonable to use an encoding technique that preserves the relation between the categories and our target variable and Target Encoding seems like a viable option. But the drawback with Target Encoding is target leakage since the target is used to predict the target. Such models tend to be overfitted and don’t generalize well in unseen circumstances.

Thus we can do one better than this and opt for CatBoost encoding. A CatBoost encoder is similar to target encoding, but also involves an ordering principle in order to overcome the problem of target leakage. It uses the principle similar to the time series data validation. The values of target statistic rely on the observed history, i.e, target probability for the current feature is calculated only from the rows (observations) before it.

Note: This may lead to a problem if our observations have an inherent ordering but since there is no inherent ordering in our data observations we already have a randomly shuffled dataset and thus CatBoost encoder should do a fine job.

We shall be implementing this technique once we have split our dataset into development and test in the later stages.

In [6]:
#CODE

#from category_encoders.cat_boost import CatBoostEncoder

#q11_encoder = CatBoostEncoder()
#X_dev['q11'] = q11_encoder.fit_transform(X_dev['q11'], y_dev)
#X_test['q11'] = q11_encoder.transform(X_test['q11'])

### Q12

In [7]:
for i in range(1, 5):
    print(f"The {df_q11_12_14_15.columns[i]} column has {df_q11_12_14_15.iloc[:,i].isnull().sum()} null values")

The q12_part_1 column has 11726 null values
The q12_part_2 column has 19076 null values
The q12_part_3 column has 12145 null values
The q12_other column has 19370 null values


These values don't signify missing data but in fact what we have in actuality is sparse data. Thus, we shall fill these null values with 0

In [8]:
for i in range(1, 5):
    df_q11_12_14_15[df_q11_12_14_15.columns[i]].fillna(0, inplace = True)

#### Method for encoding

##### Motivation

For this particular question, the most suitable encoding technique would again be One-Hot-Encoding since each of the observation can have multiple choices from the available options and through understanding of the problem statement, we feel that this might be an important feature and our goal would be to prevent any data leakage at all.

To carry this out for this dataset is fairly easy since the dataset in itself has a seperate column for each of the options within the question. All we need to do is simply convert the string data in each of the columns to 1 given we have already assigned to 0 to samples that didn't have the particular option selected.

In [9]:
# Encoding for Question 12

for i in range(1, 5):
    df_q11_12_14_15[df_q11_12_14_15.columns[i]].mask(df_q11_12_14_15[df_q11_12_14_15.columns[i]] != 0, 1 , inplace=True )
    

The column 'q12_part_3' which corresponds to the choice 'None' doesn't really add much information thus we should drop it from our dataset

In [10]:
df_q11_12_14_15.drop('q12_part_3', axis = 1, inplace = True)

### Q14

In [11]:
for i in range(4, 16):
    print(f"The {df_q11_12_14_15.columns[i]} column has {df_q11_12_14_15.iloc[:,i].isnull().sum()} null values")

The q14_part_1 column has 7694 null values
The q14_part_2 column has 11215 null values
The q14_part_3 column has 15906 null values
The q14_part_4 column has 15916 null values
The q14_part_5 column has 18899 null values
The q14_part_6 column has 19207 null values
The q14_part_7 column has 19803 null values
The q14_part_8 column has 19116 null values
The q14_part_9 column has 19186 null values
The q14_part_10 column has 19440 null values
The q14_part_11 column has 18139 null values
The q14_other column has 19471 null values


Again, these values don't signify missing data but in fact what we have in actuality is sparse data. Thus, we shall fill these null values with 0

In [12]:
for i in range(4, 16):
    df_q11_12_14_15[df_q11_12_14_15.columns[i]].fillna(0, inplace = True)

#### Method for encoding

##### Motivation

For this particular question, the most suitable encoding technique would again be One-Hot-Encoding since each of the observation can have multiple choices from the available options and through understanding of the problem statement, we feel that this might be an important feature and our goal would be to prevent any data leakage at all. We implement this in te same way as we did for Q12

In [13]:
# Encoding for Question 14

for i in range(4, 16):
    df_q11_12_14_15[df_q11_12_14_15.columns[i]].mask(df_q11_12_14_15[df_q11_12_14_15.columns[i]] != 0, 1 , inplace=True )

Again, column 'q14_part_11' which corresponds to the choice 'None' doesn't really add much information thus we should drop it from our dataset

In [14]:
df_q11_12_14_15.drop('q14_part_11', axis = 1, inplace = True)

### Q15

In [15]:
print(f"The column has {df_q11_12_14_15['q15'].isnull().sum()} null values")

The column has 3662 null values


For this question, we will be interpreting the null values (associated with no response - empty entry in the datasheet) to be no experience in using machine learning methods, thus we will impute the missing values with 'I do not use machine learning methods'

In [16]:
df_q11_12_14_15['q15'].fillna("I do not use machine learning methods", inplace = True)

In [17]:
df_q11_12_14_15['q15'].value_counts()

Under 1 year                             6312
I do not use machine learning methods    5737
1-2 years                                3459
2-3 years                                1631
3-4 years                                 893
5-10 years                                801
4-5 years                                 784
10-20 years                               244
20 or more years                          175
Name: q15, dtype: int64

#### Method for encoding

##### Motivation

Since data values are representing a range of values, we may define it as the mean of the range but the problem arises with the last category because it states 20 or more years and we have no information on the upperbound. Another technique would be to one-hot encode which makes a lot of sense but again, we have to keep in mind that we are already dealing with high-dimensional data and adding more dimensions will only make it difficult for us. But, if we observe more keenly, this column appears to have some ordinaliity since we may explicitly define an order as follows:

    1. I do not use machine learning methods
    2. Under 1 year
    3. 1-2 years
    4. 2-3 years
    5. 3-4 years
    6. 4-5 years
    7. 5-10 years
    8. 10-20 years
    9. 20 or more years
    
Thus, we shall be performing one-hot encoding for this column. This will be implemented once we have split our dataset into development and test

In [18]:
#CODE

#from sklearn.preprocessing import OrdinalEncoder

#q15_enc = OrdinalEncoder(categories=['I do not use machine learning methods','Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years', '5-10 years', '10-20 years', '20 or more years'])
#X_dev['q15'] = q15_enc.fit_transform(X_dev['q15'])
#X_test['q15'] = q15_enc.transform(X_test['q15']